# **Rent Prices & Nearby Venues Data Analysis of Munich


In [ ]:
Determine where is the best to leave in Munich (depending on the cost and nearby venues)
Segmenting and Clustering Venues in Neighborhoods of Munich's
assumption: the person like this specific kind of place and he is looking for a mid HPI
Dataset:yy

Part 1:
Part 2:
Part 3:

## PART 1/DATA SCRAPING AND CLEANING

### DATASET 1 - Postal Code, Latitude and Longitude

In [397]:
#Install required Library
import pandas as pd
import numpy as np
import re #Python RegEx

In [414]:
#Get the data from geonames website into Pandas DF
dfs = pd.read_html('https://www.geonames.org/postal-codes/DE/BY/bayern.html')
df=dfs[2] # index 2 to import only first table from the list
df.head()

,Unnamed: 0,Place,Code,Country,Admin1,Admin2,Admin3,Admin4
0,1.0,MÃ¼nchen,80331,Germany,Bayern,Upper Bavaria,Kreisfreie Stadt MÃ¼nchen,"MÃ¼nchen, Landeshauptstadt"
1,NaN,48.135/11.571,48.135/11.571,48.135/11.571,48.135/11.571,48.135/11.571,48.135/11.571,48.135/11.571
2,2.0,MÃ¼nchen,80333,Germany,Bayern,Upper Bavaria,Kreisfreie Stadt MÃ¼nchen,"MÃ¼nchen, Landeshauptstadt"
3,NaN,48.145/11.567,48.145/11.567,48.145/11.567,48.145/11.567,48.145/11.567,48.145/11.567,48.145/11.567
4,3.0,MÃ¼nchen,80335,Germany,Bayern,Upper Bavaria,Kreisfreie Stadt MÃ¼nchen,"MÃ¼nchen, Landeshauptstadt"


In [415]:
#group even and odd rows
df = df.groupby(df.index // 2).agg(lambda x: x.dropna().astype(str).str.cat(sep=' '))

#drop not necessary columns
df = df.drop(columns=['Unnamed: 0', 'Country', 'Admin1', 'Admin2', 'Admin3', 'Admin4'])

# replace the matching strings 
df = df.replace(to_replace ='MÃ¼nchen', value = 'Munich', regex = True)
df = df.replace(to_replace ='/', value = ' ', regex = True)

# extract coordinates and create new column:'coordinates'
temp_df1 = df["Place"].str.split(" ", n = 2, expand = True) 
temp_df2 = df["Code"].str.split(" ", n = 2, expand = True)
df["City"]= temp_df1[0] 
df["Latitude"]= temp_df1[1]
df["Longitude"]= temp_df1[2]
df["Postal Code"]= temp_df2[0] 
df = df[["City","Postal Code","Latitude","Longitude"]]
df


#keep only Munich
df = df[df.City == 'Munich']
df= df[['City','Postal Code','Latitude','Longitude']]
df

,City,Postal Code,Latitude,Longitude
0,Munich,80331,48.135,11.571
1,Munich,80333,48.145,11.567
2,Munich,80335,48.143,11.555
3,Munich,80336,48.135,11.559
4,Munich,80337,48.122,11.545
5,Munich,80339,48.139,11.534
6,Munich,80469,48.13,11.573
7,Munich,80538,48.14,11.588
8,Munich,80539,48.15,11.583
9,Munich,80636,48.156,11.541


### DATASET 2 - Average District Monthly Rents and Postal Code

In [421]:
# Dataset 2 contains combined dataset temp_1 and temp_2 dowloaded from the project github repository.

#dataset1
#original source: https://www.statista.com/statistics/800552/rental-costs-in-munich-germany-by-district/

#dataset2
#original source: https://www.muenchen.de/int/en/living/postal-codes.html


In [416]:
#Download CSV file from github repository: Average_apart_rent_Munich.csv
url_temp_1='https://raw.githubusercontent.com/matborg83/Coursera_Capstone_Munich/main/Average_apart_rent_Munich.csv'
temp_1 = pd.read_csv(url_temp_1)

temp_1 = temp_1.replace(to_replace ='Muchen', value = 'Munich', regex = True) #typo error in CSV file

# Combine 'Neuhausen' and 'Nymphenburg' and calculate the mean price (to match value 'Nymphenburg' in temp_2)
temp_1 = temp_1.replace(to_replace ='Neuhausen', value = 'Nymphenburg', regex = True)
temp_1 = temp_1.groupby(['District'],as_index=False).agg({'Monthly rent (EUR/Sq meter)': 'mean'})
temp_1= temp_1.replace(to_replace ='Nymphenburg', value = 'Neuhausen-Nymphenburg', regex = True) 

temp_1 = temp_1.round({'Monthly rent (EUR/Sq meter)': 2})

temp_1

,District,Monthly rent (EUR/Sq meter)
0,Altstadt-Lehel,26.57
1,Au-Haidhausen,22.92
2,Bogenhausen,21.17
3,Harlaching,20.89
4,Ludwigsvorstadt-Isarvorstadt,24.49
5,Maxvorstadt,23.45
6,Milbertshofen-Am Hart,22.72
7,Munich,20.37
8,Neuhausen-Nymphenburg,19.99
9,Pasing,21.46


In [418]:
#Download CSV file from github repository: Postal_Code_Munich(3).csv
url_temp_2='https://raw.githubusercontent.com/matborg83/Coursera_Capstone_Munich/main/Postal_Code_Munich(3).csv'
temp_2= pd.read_csv(url_temp_2)

# Normalizing district values to match dataset temp_1
temp_2.loc[85:100,'district'] = 'Schwabing,Schwabing-West'
temp_2 = temp_2.replace(to_replace ='Untergiesing-Harlaching', value = 'Harlaching', regex = True)
temp_2.loc[103:114,'district'] = 'Sendling,Sendling Westpark'
temp_2 = temp_2.replace(to_replace ='Pasing-Obermenzing', value = 'Pasing', regex = True)


# 10 districts are missing in temp_1 and will be dropped in temp_2 : example: Moosach, Obergiesing
# As a result 78 Postal code out of 128 contains rent average prices
# We could have merge the value price from "Munich" to the 10 missing districts but it would be rather innacurate to do so.
# Instead beautifulsoup could be have been used to scrap specialised websites for rents in Munich and find the missing average prices.

temp_2

,district,Postal Code
0,Allach-Untermenzing,80995
1,Allach-Untermenzing,80997
2,Allach-Untermenzing,80999
3,Allach-Untermenzing,81247
4,Allach-Untermenzing,81249
5,Altstadt-Lehel,80331
6,Altstadt-Lehel,80333
7,Altstadt-Lehel,80335
8,Altstadt-Lehel,80336
9,Altstadt-Lehel,80469


In [410]:
df2 = pd.merge(left=temp_1, right=temp_2, left_on='District', right_on='district')
df2= df2[['Postal Code','District','Monthly rent (EUR/Sq meter)']]
df2

,Postal Code,District,Monthly rent (EUR/Sq meter)
0,80331,Altstadt-Lehel,26.57
1,80333,Altstadt-Lehel,26.57
2,80335,Altstadt-Lehel,26.57
3,80336,Altstadt-Lehel,26.57
4,80469,Altstadt-Lehel,26.57
5,80538,Altstadt-Lehel,26.57
6,80539,Altstadt-Lehel,26.57
7,81541,Au-Haidhausen,22.92
8,81543,Au-Haidhausen,22.92
9,81667,Au-Haidhausen,22.92


### DATASET 3 - All data combined

In [462]:
df["Postal Code"] = df["Postal Code"].astype(str).astype(int)
df3 = pd.merge(left=df, right=df2, left_on='Postal Code', right_on='Postal Code')
df3= df3[['Postal Code','District','Monthly rent (EUR/Sq meter)','Latitude','Longitude']]

df3

,Postal Code,District,Monthly rent (EUR/Sq meter),Latitude,Longitude
0,80331,Altstadt-Lehel,26.57,48.135,11.571
1,80333,Altstadt-Lehel,26.57,48.145,11.567
2,80333,Maxvorstadt,23.45,48.145,11.567
3,80335,Altstadt-Lehel,26.57,48.143,11.555
4,80335,Ludwigsvorstadt-Isarvorstadt,24.49,48.143,11.555
5,80335,Maxvorstadt,23.45,48.143,11.555
6,80336,Altstadt-Lehel,26.57,48.135,11.559
7,80336,Ludwigsvorstadt-Isarvorstadt,24.49,48.135,11.559
8,80336,"Sendling,Sendling Westpark",20.26,48.135,11.559
9,80337,Ludwigsvorstadt-Isarvorstadt,24.49,48.122,11.545


In [461]:
# Concat Postal Code to create new unique Postal Code with combined Districts and average monthly rent price 
temp_3 = df3.sort_values(['Postal Code','District']).groupby('Postal Code', sort=False).District.apply('//'.join).reset_index(name='Combined Districts') 
temp_4= df3.groupby(['Postal Code'],as_index=False).agg({'Monthly rent (EUR/Sq meter)': 'mean'})
temp_5 = pd.merge(left=temp_3, right=temp_4, left_on='Postal Code', right_on='Postal Code')

df3 = pd.merge(left=temp_5, right=df, left_on='Postal Code', right_on='Postal Code')
df3 = df3.drop(['City'], axis=1)
df3

,Postal Code,Combined Districts,Monthly rent (EUR/Sq meter),Latitude,Longitude
0,80331,Altstadt-Lehel,26.570000,48.135,11.571
1,80333,Altstadt-Lehel//Maxvorstadt,25.010000,48.145,11.567
2,80335,Altstadt-Lehel//Ludwigsvorstadt-Isarvorstadt//...,24.836667,48.143,11.555
3,80336,Altstadt-Lehel//Ludwigsvorstadt-Isarvorstadt//...,23.773333,48.135,11.559
4,80337,"Ludwigsvorstadt-Isarvorstadt//Sendling,Sendlin...",22.375000,48.122,11.545
5,80469,Altstadt-Lehel//Ludwigsvorstadt-Isarvorstadt//...,23.773333,48.13,11.573
6,80538,"Altstadt-Lehel//Schwabing,Schwabing-West",24.515000,48.14,11.588
7,80539,Altstadt-Lehel//Maxvorstadt,25.010000,48.15,11.583
8,80634,Neuhausen-Nymphenburg,19.990000,48.149,11.533
9,80636,Maxvorstadt//Neuhausen-Nymphenburg,21.720000,48.156,11.541


## PART 2/DATA VISUALISATION

### Part 1 - Clustering and plotting the Postal Code of Munich

### Part 2 - Average District Monthly Rents and Postal Code

### Part 3 - Summary map